In [ ]:
library(haven)

hn16 <- read_sas("hn16_all.sas7bdat")
hn17 <- read_sas("hn17_all.sas7bdat")
hn18 <- read_sas("hn18_all.sas7bdat")

: Error:
there is no package called ‘haven’

In [ ]:
library(tidyverse)

cleaning_data <- function(n) {
  # 변수 이름 생성 및 데이터 가져오기
  source_name <- paste0("hn", n)
  result_name <- paste0("hn", n, "_all")
  # get()으로 데이터 가져오기
  data <- get(source_name)
  
  # 데이터 처리
  result <- data %>%
    # 19세 초과 필터링 (먼저 필터링하여 처리할 데이터 양 감소)
    filter(age > 19) %>%
    mutate(
      # 연령대 구분 명확화
      agegp = case_when(
        age >= 20 & age <= 29 ~ 0,
        age >= 30 & age <= 39 ~ 1,
        age >= 40 & age <= 49 ~ 2,
        age >= 50 & age <= 59 ~ 3,
        age >= 60 ~ 4,
        TRUE ~ NA_real_  # 명시적으로 NA_real_ 사용
      ),
      # 흡연 상태 구분
      smk = case_when(
        BS3_1 %in% c(1, 2) ~ 0,  # 비흡연
        BS3_1 == 3 ~ 2,          # 과거 흡연
        BS3_1 == 8 ~ 1,          # 현재 흡연
        TRUE ~ NA_real_
      ),
      # 음주 상태 구분
      dnk = case_when(
        BD1_11 %in% c(1, 2, 8) ~ 0,  # 비음주
        BD1_11 %in% 3:6 ~ 1,         # 음주
        TRUE ~ NA_real_
      ),
      # NA 처리 및 값 변환 (BM1 변수들)
      across(c(BM1_1, BM1_2, BM1_3, BM1_4, BM1_5, BM1_6, BM1_7, BM1_8), 
             ~ case_when(
               . %in% c(8, 9) ~ NA_real_,  # 8과 9 모두 NA로 처리
               TRUE ~ as.numeric(.)
             ))
    ) %>%
    # rowSums로 각 행의 합계 계산
    mutate(
      teethBrushing = ifelse(
        rowSums(select(., BM1_1, BM1_2, BM1_3, BM1_4, BM1_5, BM1_6, BM1_7, BM1_8) >= 1, na.rm = TRUE) >= 3, 
        1, 0
      ),
      examine = case_when(
        OR1_2 == 0 ~ 0,
        OR1_2 == 1 ~ 1,
        TRUE ~ NA_real_
      ),
      # NA 처리 (BM2 변수들)
      across(c(BM2_1, BM2_2, BM2_3, BM2_4, BM2_5), 
             ~ case_when(
               . %in% c(8, 9) ~ NA_real_,
               TRUE ~ as.numeric(.)
             ))
    ) %>%
    mutate(
      usageOfTeeth = ifelse(
        rowSums(select(., BM2_1, BM2_2, BM2_3, BM2_4, BM2_5) >= 1, na.rm = TRUE) >= 1, 
        1, 0
      ),
      preventitive = case_when(
        MO4_17 == 0 ~ 0,
        MO4_17 == 1 ~ 1,
        TRUE ~ NA_real_
      ),
      DTP = case_when(
        O_DTP >= 1 ~ 1,
        O_DTP == 0 ~ 0,
        TRUE ~ NA_real_
      )
    ) %>%
    # CPI 계산 부분 수정 - 안전하게 처리
    mutate(
      CPI = case_when(
        # 모든 값이 NA인 경우
        is.na(O_CPI_UR) & is.na(O_CPI_UM) & is.na(O_CPI_UL) & 
          is.na(O_CPI_LR) & is.na(O_CPI_LM) & is.na(O_CPI_LL) ~ NA_real_,
        # 최대값 계산 시 NA 안전하게 처리
        TRUE ~ {
          cpi_values <- c(O_CPI_UR, O_CPI_UM, O_CPI_UL, O_CPI_LR, O_CPI_LM, O_CPI_LL)
          if(all(is.na(cpi_values))) {
            NA_real_
          } else {
            max_cpi <- max(cpi_values, na.rm = TRUE)
            if(max_cpi %in% c(0, 1, 2)) 0 else if(max_cpi %in% c(3, 4)) 1 else NA_real_
          }
        }
      )
    ) %>%
    # 필요한 변수 선택
    select(
      ID, year, psu, kstrata, wt_itvex,
      sex, agegp, edu, ho_incm,
      smk,            # 흡연
      dnk,            # 음주
      teethBrushing,  # 칫솔질
      examine,        # 검진
      usageOfTeeth,   # 치아 사용
      preventitive,   # 예방
      DTP,            # 치아 문제
      CPI             # 치주 상태
    )
  
  # 결과 변수 할당 및 반환
  assign(result_name, result, envir = .GlobalEnv)
  return(result)
}


In [26]:
cleaning_data(16)
cleaning_data(17)
cleaning_data(18)ㅊㅁ

: [1m[33mError[39m in `mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `DTP = case_when(O_DTP >= 1 ~ 1, O_DTP == 0 ~ 0, TRUE ~
  NA_real_)`.
[1mCaused by error in `case_when()`:[22m
[1m[22m[33m![39m Failed to evaluate the left-hand side of formula 1.
[1mCaused by error:[22m
[33m![39m object 'O_DTP' not found

In [ ]:
hn23_1 <- hn23_filtered %>% 
  select(ID, year, region, psu, sex, age, ainc_1, ho_incm, edu, wt_itvex, kstrata, 
         BD1_11, # 음주 
         BS3_1, # 흡연
         O_TC, # 자연치아 수
         OR1, # 본인 인지 구강건강상태
         O_chew_d, # 저작 불편감
         BP1, # 평소 스트레스 인지 정도
         BP5 # 평소 2주이상 우울감 여부
  )

hn22_1 <- hn22_filtered %>% 
  select(ID, year, region, psu, sex, age, ainc_1, ho_incm, edu, wt_itvex, kstrata, 
         BD1_11, # 음주 
         BS3_1, # 흡연
         O_TC, # 자연치아 수
         OR1, # 본인 인지 구강건강상태
         O_chew_d, # 저작 불편감
         BP1, # 평소 스트레스 인지 정도
         BP5 # 평소 2주이상 우울감 여부
    )

# 데이터 결합
hn22_23 <- rbind(hn22_1, hn23_1)

In [ ]:
library(labelled)

hn22_23_final <- hn22_23 |> 
  mutate(
    smk = case_when( 
      BS3_1 %in% 1:2 ~ 1,
      BS3_1 %in% c(3,8) ~ 0,
      TRUE ~ NA),
    dnk = case_when(
      BD1_11 %in% 1:3 ~ 0,
      BD1_11 %in% 4:8 ~ 1,
      TRUE ~ NA),
    age_gp = case_when(
      age %in% 19:29 ~ 0,
      age %in% 30:39 ~ 1,
      age %in% 40:49 ~ 2,
      age %in% 50:64 ~ 3,
      age >= 65 ~ 4,
      TRUE ~ NA), # NA가 문자형일 때
    dep = case_when(
      BP5 == 1 ~ 1,
      BP5 %in% c(2,8) ~ 0,
      TRUE ~ NA),
    stress = case_when(
      BP1 %in% 1:2 ~ 1,
      BP1 %in% 3:4 ~ 0,
      TRUE ~ NA),
    wt = wt_itvex / 2 # 가중치
  ) |> 
  rename(
    id = ID,
    nt = O_TC,
    srh = OR1,
    chew = O_chew_d) |>
  mutate(across(c(year, sex, age_gp, region, ho_incm, edu, smk, dnk, chew, srh, stress, dep), as.factor)
  ) |> 
  drop_na()

In [ ]:
if(!require(skimr)) install.packages("skimr", dependencies = TRUE)
library(skimr)

hn22_23_final %>% skim()

In [ ]:
hn22_23_final |> 
  arrange(stress) |> # 오름차순
  group_by(stress) |> # 그룹별
  slice(1)

In [ ]:
if(!require(gtsummary)) install.packages("gtsummary", dependencies = TRUE)
if(!require(labelled)) install.packages("labelled")
library(gtsummary)

table1 <- hn22_23_final |> 
  select(year, sex, age_gp, ho_incm, edu, smk, dnk, nt, srh, 
         chew, stress, dep) |> 
  tbl_summary(
            by=stress, 
            statistic = list(all_categorical() ~ "{n} ({p}%)", 
                             all_continuous() ~ "{mean} ± {sd}")
  ) |> 
  add_p()

as_hux_xlsx(table1, "table1.xlsx")

In [ ]:
# 1) 정규성 검정
# 정규성 검정: Shapiro-Wilk test # 샘플 수가 5000개 이하만 가능
set.seed(2015)  # set.seed는 랜덤하게 sampling할 경우 재현가능성을 위해 쓰임.
nt_sample_0 <- sample(hn22_23_final$nt[hn22_23_final$stress == "0"], 5000)
shapiro.test(nt_sample_0)

shapiro.test(hn22_23_final$nt[hn22_23_final$stress == "1"])

# 2) 등분산성 검정: 기본적으로 var.test()

var.test(nt ~ stress, data=hn22_23_final)

In [ ]:
# 독립표본 t-검정

# 등분산 가정할 경우 - 독립표본 t-검정 실시
t.test(nt ~ stress, data = hn22_23_final, var.equal = TRUE)

# 이분산 가정할 경우 - Welch's t-test
t.test(nt ~ stress, data = hn22_23_final, var.equal = FALSE)

In [ ]:
# Wilcoxon rank-sum test 실행
wilcox_result <- wilcox.test(nt ~ stress, data = hn22_23_final)

In [ ]:
if(!require(car)) install.packages("car", dependencies = TRUE)
library(car)
set.seed(123)

# ANOVA 실행 전, 잔차 추출
sampled_data <- hn22_23_final[sample(nrow(hn22_23_final), 5000), ]
anova_model <- aov(nt ~ ho_incm, data = sampled_data)
residuals_anova <- residuals(anova_model)

# 정규성 검정: Shapiro-Wilk test on residuals
shapiro_anova <- shapiro.test(residuals_anova)

# 분산 동질성 검정: Levene's test (car 패키지 필요)
levene_result <- leveneTest(nt ~ ho_incm, data = hn22_23_final)

In [ ]:
# ANOVA 실행
anova_result <- summary(anova_model)

In [ ]:
# Kruskal-Wallis test 실행
kruskal_result <- kruskal.test(nt ~ ho_incm, data = hn22_23_final)

In [ ]:
# 교차분할표 생성
table_cat <- table(hn22_23_final$stress, hn22_23_final$chew)

# 카이제곱 검정 실행
chisq.test(table_cat)

# 기대빈도 확인: 기대빈도란 귀무가설이 참일 경우, 각 셀에 기대되는 이론적인 빈도
chisq.test(table_cat)$chisq_result$expected

In [ ]:
fisher.test(table_cat)

In [ ]:
library(car)

# 로지스틱 회귀 모델
model <- glm(as.numeric(stress) ~ nt + chew + srh + sex +
                age_gp + ho_incm + edu + smk + dnk, data = hn22_23_final, na.action = na.omit)

# VIF 확인: Variance Inflation Factor
vif(model)

In [ ]:
# 복합표본 설계 객체 만들기
if(!require(survey)) install.packages("survey", dependencies = TRUE)
if(!require(gt)) install.packages("gt", dependencies = TRUE)

library(survey)
library(gt)

dstrat <- svydesign(id = ~id, strata = ~kstrata, weights = ~wt, data = hn22_23_final)

# 로지스틱 회귀모형 적합
fit <- svyglm(stress ~ nt + chew + srh + sex +
              age_gp + ho_incm + edu + smk + dnk,
              design = dstrat, family = quasibinomial())
summary(fit)

table_multiLogistic1 <- fit |> 
   tbl_regression(exponentiate = TRUE)

# 로지스틱 회귀모형 적합
fit2 <- svyglm(dep ~ nt + chew + srh + sex +
               age_gp + ho_incm + edu + smk + dnk,
               design = dstrat, family = quasibinomial())
summary(fit2)

table_multiLogistic2 <- fit2 |> 
   tbl_regression(exponentiate = TRUE)


merged_table <- tbl_merge(
   tbls = list(table_multiLogistic1, table_multiLogistic2),
   tab_spanner = c("**Model1**", "**Model2**") # 각 표의 머리글(스패너) 지정(선택)
 )

 as_hux_xlsx(merged_table, "table2.xlsx")